In [1]:
setwd(fs::path_abs("~/Local_Workspace/TesiMag"))
library(dplyr, warn.conflicts = FALSE)
library(arrow, warn.conflicts = FALSE)
library(duckdb, warn.conflicts = FALSE)
library(DBI, warn.conflicts = FALSE)

path.lom <- file.path("/Users/davidenicoli/Local_Workspace/Datasets/ARPA/LOMBARDIA")

Caricamento del pacchetto richiesto: DBI



In [2]:
open_orig_ds.arpalombardia <- function() {
    path.fragments <- file.path(path.lom, "fragments")

    read_conf <- list(
        "schema" = schema(
            IdSensore = int32(),
            Data = timestamp(unit = "s"),
            Valore = float64(),
            Stato = utf8(),
            idOperatore = int8()
        ),
        "ts_parsers" = csv_timestamp_parsers <- c("%d/%m/%Y %H:%M:%S", "%d/%m/%Y %H:%M:%S.000"),
        "nas" = na_strings <- c("", "NA", "9999.0", "-9999.0", "-9999", "NV", "NC", "ND")
    )

    open_csv_dataset(fs::dir_ls(path.fragments, glob = "*.csv"),
        hive_style = FALSE,
        schema = read_conf$schema,
        timestamp_parsers = read_conf$ts_parsers,
        na = read_conf$nas,
        col_names = TRUE,
        skip = 1
    ) |>
        rename(station_id = IdSensore, time = Data, value = Valore)
}

In [7]:
conn <- dbConnect(duckdb::duckdb(), file.path(path.lom,"lombardia.db"))
dbWriteTableArrow(conn, "full_tmp", open_orig_ds.arpalombardia())

In [8]:
gc()

,used,(Mb),gc trigger,(Mb),limit (Mb),max used,(Mb)
Ncells,1388814,74.2,2874520,153.6,NA,2499901,133.6
Vcells,2564212,19.6,8982499,68.6,16384,8982499,68.6


In [3]:
library(lubridate, warn.conflicts = FALSE)
open_orig_ds.arpalombardia() |> write_dataset(file.path(path.lom, "dataset"))

In [4]:
open_orig_ds.arpalombardia()

FileSystemDataset (query)
station_id: int32
time: timestamp[s]
value: double
Stato: string
idOperatore: int8

See $.data for the source Arrow object